In [1]:
import pickle
sentimentAnal = pickle.load(open('twitterSentimentMA.sav','rb'))

In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("csvNewResults.csv", encoding = 'latin-1')
# above line will be different depending on where you saved your data, and your file name
df.drop(['id','keyword','handle','replies','retweets','likes'],axis=1,inplace=True)
df.head(10)


,body,date_tweeted
0,Snapchat so worthless now,2018-07-24T04:01:00.000Z
1,snapchatâs lame just text me,2018-07-24T04:12:37.000Z
2,if your bff doesnât share her/his location w...,2018-07-24T04:14:42.000Z
3,"Doing shit for Twitter, Instagram, Snapchat, o...",2018-07-24T04:19:46.000Z
4,your mcm typed out âViolence isnât the ans...,2018-07-24T04:23:22.000Z
5,"âWomen......, please treat your man better. ...",2018-07-24T04:28:28.000Z
6,"Dear Snapchat, please apply a âall streaksâ...",2018-07-24T04:32:24.000Z
7,Luis is big mad I lost our Snapchat streak LMA...,2018-07-24T04:47:48.000Z
8,Remember back in the good old days when you cl...,2018-07-24T04:49:21.000Z
9,canât keep a snapchat streak because im not ...,2018-07-24T04:50:12.000Z


In [3]:
from bs4 import BeautifulSoup
import re
df['pre_clean_len'] = [len(t) for t in df.body]

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(body):
    soup = BeautifulSoup(body, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()
testing = df.body[:100]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result


['snapchat so worthless now',
 'snapchat s lame just text me',
 'if your bff doesn t share her his location with you on snapchat is she he really your bff haha',
 'doing shit for twitter instagram snapchat or facebook doesn t get you nowhere in real life',
 'your mcm typed out violence isn t the answer on a black screen for his snapchat story and thought he did something',
 'women please treat your man better i m tired of getting deleted then re added on snapchat',
 'dear snapchat please apply a all streaks button on the next update',
 'luis is big mad i lost our snapchat streak lmaoohmuforstreaksooo pic twitter com didrohxzwo',
 'remember back in the good old days when you clicked through snapchat stories for entertainment instead of just clearing them out of annoyance ahhhh',
 'can t keep a snapchat streak because im not committed to anything',
 'if you don t like what i post on snapchat please remove me you won t be missed',
 'the flashback memories on snapchat give me bad vibes bc 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16894 entries, 0 to 16893
Data columns (total 3 columns):
body             16894 non-null object
date_tweeted     16894 non-null object
pre_clean_len    16894 non-null int64
dtypes: int64(1), object(2)
memory usage: 396.0+ KB


In [5]:
nums = [0,16894]
print ("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_tweet_texts.append(tweet_cleaner(df['body'][i]))

Cleaning and parsing the tweets...

Tweets 10000 of 16894 has been processed


In [6]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['body'])

In [7]:
clean_df.head()

,body
0,snapchat so worthless now
1,snapchat s lame just text me
2,if your bff doesn t share her his location wit...
3,doing shit for twitter instagram snapchat or f...
4,your mcm typed out violence isn t the answer o...


In [8]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [9]:
clean_df.head()

,body
0,snapchat so worthless now
1,snapchat s lame just text me
2,if your bff doesn t share her his location wit...
3,doing shit for twitter instagram snapchat or f...
4,your mcm typed out violence isn t the answer o...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(min_df = 1)
cvec.fit(clean_df.body)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [23]:
len(cvec.get_feature_names())
clean_df.to_csv('clean_data.csv',encoding='utf-8')

In [20]:
document_matrix = cvec.transform(clean_df.body)

clean_df_new = cvec.transform(clean_df.body)

In [21]:
sentimentAnal.predict(clean_df_new)

ValueError: X has 28192 features per sample; expecting 10572579

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df= 1, analyzer = "word")

In [18]:
vect.fit(clean_df)
clean_df_dtm = vect.transform(clean_df) 

In [19]:
sentimentAnal.predict(clean_df_dtm)

ValueError: X has 1 features per sample; expecting 10572579

In [ ]:
pred = sentimentAnal.predict(clean_df.body)

In [ ]:
sen = ['girl is so ugly', 'im hungry']

sentimentAnal.predict(sen)

In [24]:
sentimentAnal.predict(["hello this sucks"])

ValueError: Expected 2D array, got 1D array instead:
array=['hello this sucks'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.